Source : https://analyticsindiamag.com/my-first-cnn-project-emotion-detection-using-convolutional-neural-network-with-tpu/

Emotion detection using CNN 

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import keras
import keras.utils
from keras import utils as np_utils

In [2]:
emotion_data = pd.read_csv('/Users/romai/Documents/Ecole/Ingé - M1_auto/Q2/Système intelligent/fer2013.csv')
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [3]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

In [4]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

In [5]:
print('X_train : ',X_train.shape)
print('y_train : ',y_train.shape)
print('X_test  : ',X_test.shape)
print('y_test  : ',y_test.shape)

X_train :  (28709, 48, 48, 1)
y_train :  (28709, 7)
X_test  :  (3589, 48, 48, 1)
y_test  :  (3589, 7)


In [36]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)

print('Before normalization : Min={}, max={}'.format(X_train.min(),X_train.max()))

xmax=X_train.max()
X_train = X_train / xmax
X_test  = X_test  / xmax

print('After normalization  : Min={}, max={}'.format(X_train.min(),X_train.max()))

Before normalization : Min=0.0, max=1.0
After normalization  : Min=0.0, max=1.0


In [37]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2),padding='same', strides=(2,2)))

In [38]:
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2), padding='same', strides=(2,2)))

In [39]:
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same', strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same', strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same', strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [41]:
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [42]:
model.fit(X_train, y_train, batch_size=512, epochs=1, verbose=1, validation_data = (X_test, y_test))

57/57 [==============================] - 1352s 24s/step - loss: 1.9370 - accuracy: 0.2461 - val_loss: 1.9123 - val_accuracy: 0.2494


In [45]:
loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)

113/113 [==============================] - 5s 47ms/step - loss: 1.9123 - accuracy: 0.2494
[1.9123033285140991, 0.24937307834625244]


In [46]:
import pickle as pkl

#Save the model in knnmodel file
model = "model.pkl"
with open(model, 'wb') as file:
    pkl.dump(model, file)

# Testing model with camera

In [47]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [48]:
# Load knnmodel file
with open(model, 'rb') as file:
    model = pkl.load(file)

In [49]:
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [57]:
import tensorflow as tf
from PIL import Image

cap=cv2.VideoCapture(0)

while True:
    ret,image=cap.read(0)
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces_detected = face_haar_cascade.detectMultiScale(converted_image)
    
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(image,(x,y), (x+w,y+h), (255,0,0))
    roi_gray=image[y:y+w,x:x+h,1]
    # il faut rajouter un "1" dans les paramètres de matrice 
    roi_gray=cv2.resize(roi_gray,(48,48))
    image_pixels = tf.keras.preprocessing.image.img_to_array(roi_gray)
    image_pixels = np.expand_dims(image_pixels, axis = 0)
    image_pixels /= 255

    
    
    predictions = model.predict(image_pixels) 
    max_index = np.argmax(predictions[0])

    emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    emotion_prediction = emotion_detection[max_index]
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, emotion_prediction, (int(x), int(y)), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    
    
    cv2.imshow('Demo video', image)
    resize_image = cv2.resize(image, (1000, 700))
    cv2.imshow('Emotion',resize_image)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

    print ('image :', np.shape(image_pixels))
    print ('model :', np.shape(X_train))
cap.release()
cv2.destroyAllWindows

AttributeError: 'str' object has no attribute 'predict'